In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("OKCUPID_original.csv",encoding='utf8')

### Initial feature filtering

Keep only required columns (Non open ended category)

In [3]:
col_keep=['age','status','orientation','body_type',
          'diet','drinks','drugs','education','ethnicity',
          'height','income','job','last_online','location',
          'offspring','pets','religion','sign','smokes','speaks','sex']

df_oc=df[col_keep]

### Data cleaning

#### (A) Data types modifications

Check for data types

In [4]:
df_oc.dtypes

age              int64
status          object
orientation     object
body_type       object
diet            object
drinks          object
drugs           object
education       object
ethnicity       object
height         float64
income           int64
job             object
last_online     object
location        object
offspring       object
pets            object
religion        object
sign            object
smokes          object
speaks          object
sex             object
dtype: object

Apply the following to data types:
- Change 'last online' to date type

In [5]:
df_oc['last_online']=pd.to_datetime(df_oc['last_online'].str.slice(stop=10))

C:\Users\HP-073\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### (B) Nulls data modifications

Check for of NANs

In [6]:
print('The % of NANs per feature are:\n')
print(round(df_oc.isnull().sum()/len(df_oc)*100,2))

The % of NANs per feature are:

age             0.00
status          0.00
orientation     0.00
body_type       8.83
diet           40.69
drinks          4.98
drugs          23.49
education      11.06
ethnicity       9.48
height          0.01
income          0.00
job            13.68
last_online     0.00
location        0.00
offspring      59.32
pets           33.23
religion       33.74
sign           18.44
smokes          9.19
speaks          0.08
sex             0.00
dtype: float64


We want to avoid missing values on our model so we will:
- Replace body_type NANs with "rather not say"
- Replace height NANs with the average height
- Drop all other NANs

In [7]:
df_oc['body_type']=df_oc['body_type'].fillna('rather not say')
df_oc['height']=df_oc['height'].fillna(round(df_oc['height'].mean(),2))
#df_oc=df_oc.fillna('Missing')
df_oc=df_oc.dropna()

C:\Users\HP-073\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\HP-073\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
print('The % of NANs per feature after is:\n')
print(df_oc.isnull().sum())

The % of NANs per feature after is:

age            0
status         0
orientation    0
body_type      0
diet           0
drinks         0
drugs          0
education      0
ethnicity      0
height         0
income         0
job            0
last_online    0
location       0
offspring      0
pets           0
religion       0
sign           0
smokes         0
speaks         0
sex            0
dtype: int64


### Data preprocessing

#### (A) Modifying features

##### body_type
Set as an ordered rank

In [9]:
bt_values=['rather not say','used up','skinny','thin',
           'fit','athletic','average','jacked','curvy',
           'a little extra','overweight','full figured']

for i in range(len(bt_values)):
    df_oc.loc[df_oc['body_type']==bt_values[i], 'body_type'] = i

##### diet
Diet intensity taken away and left with the diet (anything, missing, halal, kosher, other, vegan, vegetarian)

In [10]:
diet_values=['anything','missing','halal','kosher',
             'other','vegan','vegetarian']

for i in range(len(diet_values)):
    df_oc.loc[df_oc['diet'].str.contains(pat =diet_values[i])==True, 'diet'] = diet_values[i]

##### drinks
Set as an ordered rank

In [11]:
drink_values=['not at all','rarely','socially',
              'often','very often','desperately']

for i in range(len(drink_values)):
    df_oc.loc[df_oc['drinks']==drink_values[i], 'drinks'] = i

##### drugs
Set as an ordered rank

In [12]:
drug_values=['never','sometimes','often']

for i in range(len(drug_values)):
    df_oc.loc[df_oc['drugs']==drug_values[i], 'drugs'] = i

##### education 
Group different categories based on this conversion dataframe and also differentiate if they are dropouts or not

In [13]:
educ_cats=pd.DataFrame([('college/university','college/university',0),('dropped out of college/university','college/university',1),
                        ('dropped out of high school','high school',1),('dropped out of law school','specialized program',1),
                        ('dropped out of masters program','masters program',1),('dropped out of med school','specialized program',1),
                        ('dropped out of ph.d program','ph.d program',1),('dropped out of space camp','high school',1),
                        ('dropped out of two-year college','high school',1),('graduated from college/university','college/university',0),
                        ('graduated from high school','high school',0),('graduated from law school','specialized program',0),
                        ('graduated from masters program','masters program',0),('graduated from med school','specialized program',0),
                        ('graduated from ph.d program','ph.d program',0),('graduated from space camp','high school',0),
                        ('graduated from two-year college','high school',0),('high school','high school',0),
                        ('law school','specialized program',0),('masters program','masters program',0),
                        ('med school','specialized program',0),('ph.d program','ph.d program',0),
                        ('space camp','high school',0),('two-year college','college/university',0),
                        ('working on college/university','college/university',0),('working on high school','high school',0),
                        ('working on law school','specialized program',0),('working on masters program','masters program',0),
                        ('working on med school','specialized program',0),('working on ph.d program','ph.d program',0),
                        ('working on space camp','high school',0),('working on two-year college','high school',0)],
                        columns=('education', 'educ_cat','dropped out'))   

edu_values=['high school','college/university','specialized program','masters program','ph.d program']
for i in range(len(edu_values)):
    educ_cats.loc[educ_cats['educ_cat'].str.contains(pat =edu_values[i])==True, 'educ_cat'] = i+1

display(educ_cats)

,education,educ_cat,dropped out
0,college/university,2,0
1,dropped out of college/university,2,1
2,dropped out of high school,1,1
3,dropped out of law school,3,1
4,dropped out of masters program,4,1
5,dropped out of med school,3,1
6,dropped out of ph.d program,5,1
7,dropped out of space camp,1,1
8,dropped out of two-year college,1,1
9,graduated from college/university,2,0


Replace education for the new educ_cats

In [14]:
df_oc=df_oc.merge(educ_cats, on='education', how='left')
df_oc=df_oc.drop(['education'], axis=1)
df_oc.rename(columns={'educ_cat': 'education'}, inplace=True)

##### job
Group different categories based on this conversion dataframe

In [15]:
job_cats=pd.DataFrame([('military','other'),('retired','other'),('unemployed','other'),
                       ('transportation','non-technical'),('rather not say','other'),('political / government','law and government'),
                       ('clerical / administrative','non-technical'),('construction / craftsmanship','non-technical'),('hospitality / travel','non-technical'),
                       ('law / legal services','law and government'),('entertainment / media','entertainment / media'),('banking / financial / real estate','business and management'),
                       ('executive / management','business and management'),('education / academia','education'),('medicine / health','medicine / health'),
                       ('sales / marketing / biz dev','business and management'),('artistic / musical / writer','artistic / musical / writer'),('computer / hardware / software','technology'),
                       ('science / tech / engineering','technology'),('student','education'),('other','other')],
                        columns=('job', 'job_cat'))   
display(job_cats)

,job,job_cat
0,military,other
1,retired,other
2,unemployed,other
3,transportation,non-technical
4,rather not say,other
5,political / government,law and government
6,clerical / administrative,non-technical
7,construction / craftsmanship,non-technical
8,hospitality / travel,non-technical
9,law / legal services,law and government


In [16]:
df_oc=df_oc.merge(job_cats, on='job', how='left')
df_oc=df_oc.drop(['job'], axis=1)
df_oc.rename(columns={'job_cat': 'job'}, inplace=True)

##### religion
Religion is deconstructed in two:
- religion: Categorical variable of religion
- relig_imprt: Religion intensity scale (laughing about it, not too serious about it, somewhat serious about it, very serious about it)

In [17]:
rel_values=['agnosticism','atheism','buddhism','catholicism',
             'christianity','hinduism','islam','judaism','other']

relimp_values=['laughing about it','not too serious about it',
               'somewhat serious about it','very serious about it']

for i in range(len(rel_values)):
    df_oc.loc[df_oc['religion'].str.contains(pat =rel_values[i])==True, 'religion2'] = rel_values[i]
    
for i in range(len(relimp_values)):
    df_oc.loc[df_oc['religion'].str.contains(pat =relimp_values[i])==True, 'relig_imprt'] = i
    
df_oc=df_oc.drop(['religion'], axis=1)
df_oc.rename(columns={'religion2': 'religion'}, inplace=True)

##### sign
Sign is deconstructed in two:
- sign: Categorical variable of religion
- sign_imprt: Sign intensity scale (it doesn't matter, it's fun to think about, it matters a lot)

In [18]:
sign_values=['gemini','cancer','pisces','aquarius','taurus','virgo',
             'sagittarius','leo','aries','libra','scorpio','capricorn']

signimp_values=["it doesn't matter","it's fun to think about","it matters a lot"]

for i in range(len(sign_values)):
    df_oc.loc[df_oc['sign'].str.contains(pat =sign_values[i])==True, 'sign2'] = sign_values[i]
    
for i in range(len(signimp_values)):
    df_oc.loc[df_oc['sign'].str.contains(pat =signimp_values[i])==True, 'sign_imprt'] = i
    
df_oc=df_oc.drop(['sign'], axis=1)
df_oc.rename(columns={'sign2': 'sign'}, inplace=True)

##### smokes
Set as an ordered rank

In [19]:
smokes_values=['no','trying to quit','sometimes','when drinking','yes']

for i in range(len(smokes_values)):
    df_oc.loc[df_oc['smokes']==smokes_values[i], 'smokes'] = i

##### speaks to num_spoken_languages
Count the number of languages spoken (if the was not language reported it is counted as 1)

In [20]:
df_oc['speaks']=df_oc['speaks'].apply(lambda x: len(x.split(',')))

#### (B) Dummifying complex categorical variables

##### offspring dummy (havekids, wantkids)
Deconstruct offspring variable to account if:
- People have kids
- People want kids

##### ethnicity 
Transform etnicity to dummies (asian, white, black, hispanic/ latin, pacific islander, native american, middle eastern, indian, other)

In [21]:
ethn_values= ['asian','white','black','hispanic / latin',
              'pacific islander','native american','middle eastern',
              'indian','other']

for i in range(len(ethn_values)):
    df_oc[ethn_values[i]]=0
    df_oc.loc[df_oc['ethnicity'].str.contains(pat =ethn_values[i])==True, ethn_values[i]] = 1

df_oc=df_oc.drop(['ethnicity'], axis=1)

##### offspring dummy (havekids, wantkids)
Deconstruct offspring variable to account if:
- People have kids
- People want kids

In [22]:
offs_cats=pd.DataFrame([("doesn't have kids, and doesn't want any",0,0),
                        ("doesn't want kids",np.nan,0),
                        ("has a kid, but doesn't want more",1,0),
                        ("has kids, but doesn't want more",1,0),
                        ("doesn't have kids, but might want them",0,np.nan),
                        ("doesn't have kids",0,np.nan),
                        ("might want kids",np.nan,np.nan),
                        ("has a kid","1",np.nan),
                        ("has a kid, and might want more",1,np.nan),
                        ("has kids",1,np.nan),
                        ("has kids, and might want more",1,np.nan),
                        ("doesn't have kids, but wants them",0,1),
                        ("wants kids",np.nan,1),
                        ("has a kid, and wants more",1,1),
                        ("has kids, and wants more",1,1)],
                        columns=('offspring', 'havekids','wantkids'))     
display(offs_cats)

,offspring,havekids,wantkids
0,"doesn't have kids, and doesn't want any",0,0.0
1,doesn't want kids,NaN,0.0
2,"has a kid, but doesn't want more",1,0.0
3,"has kids, but doesn't want more",1,0.0
4,"doesn't have kids, but might want them",0,NaN
5,doesn't have kids,0,NaN
6,might want kids,NaN,NaN
7,has a kid,1,NaN
8,"has a kid, and might want more",1,NaN
9,has kids,1,NaN


Add new dummy variables to database

In [23]:
df_oc=df_oc.merge(offs_cats, on='offspring', how='left')
df_oc=df_oc.drop(['offspring'], axis=1)

##### pets dummy (likes_dog, likes_cat, has_dog, has_cat)
Deconstruct offspring variable to account if:
- Likes dogs
- Likes cat
- Has dog
- Has cat

In [24]:
pets_cats=pd.DataFrame([("likes dogs and likes cats","1","1","0","0"),
                        ("has cats","0","1","0","1"),
                        ("likes cats","0","1","0","0"),
                        ("has dogs and likes cats","1","1","1","0"),
                        ("likes dogs and has cats","1","1","0","1"),
                        ("likes dogs and dislikes cats","1","0","0","0"),
                        ("has dogs","1","0","1","0"),
                        ("has dogs and dislikes cats","1","0","1","0"),
                        ("likes dogs","1","0","0","0"),
                        ("has dogs and has cats","1","1","1","1"),
                        ("dislikes dogs and has cats","0","1","0","1"),
                        ("dislikes dogs and dislikes cats","0","0","0","0"),
                        ("dislikes cats","0","0","0","0"),
                        ("dislikes dogs and likes cats","0","1","0","0"),
                        ("dislikes dogs","0","0","0","0")],
                        columns=('pets', 'likes_dog','likes_cat','has_dog','has_cat'))     
display(pets_cats)

,pets,likes_dog,likes_cat,has_dog,has_cat
0,likes dogs and likes cats,1,1,0,0
1,has cats,0,1,0,1
2,likes cats,0,1,0,0
3,has dogs and likes cats,1,1,1,0
4,likes dogs and has cats,1,1,0,1
5,likes dogs and dislikes cats,1,0,0,0
6,has dogs,1,0,1,0
7,has dogs and dislikes cats,1,0,1,0
8,likes dogs,1,0,0,0
9,has dogs and has cats,1,1,1,1


Add new dummy variables to database

In [25]:
df_oc=df_oc.merge(pets_cats, on='pets', how='left')
df_oc=df_oc.drop(['pets'], axis=1)

#### (C) Dummyfying categorical variables
The following categorical variables will be dummified:
- status
- orientation
- diet
- job
- religion
- sign

In [26]:
d_features= ['status','orientation','diet','job','religion','sign','sex']

for j in d_features:
    values=df_oc[j].unique()
    for i in range(len(values)):
        df_oc[j+'-'+values[i]]=0
        df_oc.loc[df_oc[j].str.contains(pat =values[i])==True, j+'-'+values[i]] = 1

df_oc=df_oc.drop(['sex','sex-f'], axis=1)

#### (D) Deal with final NANs

In [27]:
nan_features=['relig_imprt','sign_imprt']

for i in nan_features:
    df_oc[i]=df_oc[i].fillna(round(df_oc[i].mean(),0))

nan_features=['havekids','wantkids']

for i in nan_features:
    df_oc[i]=df_oc[i].fillna(-1)

In [28]:
pd.set_option('display.max_columns', None)
display(df_oc)

,age,status,orientation,body_type,diet,drinks,drugs,height,income,last_online,location,smokes,speaks,education,dropped out,job,religion,relig_imprt,sign,sign_imprt,asian,white,black,hispanic / latin,pacific islander,native american,middle eastern,indian,other,havekids,wantkids,likes_dog,likes_cat,has_dog,has_cat,status-single,status-available,status-seeing someone,status-married,orientation-straight,orientation-bisexual,orientation-gay,diet-anything,diet-other,diet-halal,diet-vegetarian,diet-vegan,diet-kosher,job-non-technical,job-artistic / musical / writer,job-other,job-technology,job-business and management,job-education,job-medicine / health,job-entertainment / media,job-law and government,religion-agnosticism,religion-christianity,religion-catholicism,religion-buddhism,religion-other,religion-atheism,religion-judaism,religion-hinduism,religion-islam,sign-gemini,sign-cancer,sign-sagittarius,sign-taurus,sign-pisces,sign-aquarius,sign-libra,sign-aries,sign-capricorn,sign-virgo,sign-leo,sign-scorpio,sex-m
0,22,single,straight,9,anything,2,0,75.0,-1,2012-06-28,"south san francisco, california",2,1,2,0,non-technical,agnosticism,3.0,gemini,1.0,1,1,0,0,0,0,0,0,0,0,-1.0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,35,single,straight,6,other,3,1,70.0,80000,2012-06-29,"oakland, california",0,3,1,0,non-technical,agnosticism,1.0,cancer,1.0,0,1,0,0,0,0,0,0,0,0,-1.0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,31,single,straight,6,anything,2,0,65.0,-1,2012-06-29,"san francisco, california",0,2,2,0,artistic / musical / writer,christianity,1.0,sagittarius,1.0,0,1,0,0,0,0,0,0,0,0,1.0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,29,single,straight,3,anything,2,0,62.0,50000,2012-06-29,"san leandro, california",0,1,2,0,other,catholicism,1.0,taurus,1.0,0,1,0,1,0,0,0,0,0,0,1.0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,33,single,straight,5,anything,2,0,72.0,-1,2012-06-27,"san francisco, california",0,1,4,0,technology,catholicism,1.0,pisces,1.0,0,1,0,0,0,0,0,0,0,0,-1.0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7238,24,single,straight,2,anything,2,1,72.0,20000,2012-06-29,"oakland, california",2,2,2,0,artistic / musical / writer,atheism,2.0,taurus,1.0,0,1,0,0,0,0,0,0,0,0,-1.0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
7239,29,single,straight,11,anything,2,0,64.0,-1,2012-06-30,"san francisco, california",0,2,2,0,business and management,agnosticism,0.0,libra,0.0,0,0,1,0,0,0,0,0,0,0,-1.0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7240,22,single,straight,0,anything,2,1,68.0,-1,2012-06-25,"san francisco, california",0,2,2,0,technology,agnosticism,1.0,sagittarius,0.0,1,0,0,1,0,0,0,0,0,0,-1.0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7241,24,single,straight,4,anything,3,1,72.0,-1,2012-06-29,"san francisco, california",0,1,2,0,entertainment / media,agnosticism,1.0,leo,0.0,0,1,0,0,0,0,0,0,1,0,-1.0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [29]:
df_oc.to_csv("OKCUPID_final.csv",index=False)